# Programming assignment 4: Implementing linear classifiers

| Students      | E-mail |
| ----------- | ----------- |
| Leo Carlsson      | leo@leocarlsson.se        |
| Efstathios Sidiropoulos   | efstsidiropoulos@gmail.com        |

## Introduction

In [1]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y
    
# Read all the documents.
X, Y = read_data('data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
    Perceptron()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 0.93 sec.
Accuracy: 0.7919.


This file shows a couple of implementations of the perceptron learning
algorithm. It is based on the code from Lecture 3, but using the slightly
more compact perceptron formulation that we saw in Lecture 6.

There are two versions: Perceptron, which uses normal NumPy vectors and
matrices, and SparsePerceptron, which uses sparse vectors and matrices.
The latter may be faster when we have high-dimensional feature representations
with a lot of zeros, such as when we are using a "bag of words" representation
of documents.

## Implementing SVC

In [2]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X: np.array):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X: np.array):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])



class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x

class SVM(LinearClassifier):
    def __init__(self, lambda_value=0.001, n_iter=20) -> None:
        self.n_iter = n_iter
        self.lambda_value = lambda_value

    def fit(self, X, Y):
        self.find_classes(Y)

        Ye = self.encode_outputs(Y)
    
        if not isinstance(X, np.ndarray):
            X = X.toarray()
        
        self.w = np.zeros(X.shape[1])
        t = 0
        
        #Pegasos
        for i in range(self.n_iter):
            
            for x, y in zip(X, Ye):
                t = t + 1

                eta = 1/(self.lambda_value * t)
                score = self.w @ x 
                

                if (y * score) < 1:
                    self.w = (1-eta * self.lambda_value) * self.w + (eta*y) * x
                else:
                    self.w = (1-eta * self.lambda_value) * self.w
            
            if i % 5 == 0:
                print(f"Epoch {i}")
                print(f'Score: {score:.2f}')

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    SVM()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
SVM_time = t1-t0

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Epoch 0
Score: -0.95
Epoch 5
Score: -0.77
Epoch 10
Score: -0.76
Epoch 15
Score: -0.75
Training time: 1.47 sec.
Accuracy: 0.8238.


## Implementing Logistic Regression

In [3]:
class Log_reg(LinearClassifier):
    def __init__(self, lambda_value=0.001, n_iter=20) -> None:
        self.n_iter = n_iter
        self.lambda_value = lambda_value

    def fit(self, X, Y):
        self.find_classes(Y)

        Ye = self.encode_outputs(Y)
    
        if not isinstance(X, np.ndarray):
            X = X.toarray()
        
    
        self.w = np.zeros(X.shape[1])
        t = 0

        #Pegasos
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                t = t + 1

                eta = 1/(self.lambda_value * t)
                score = self.w @ x 
                gradient = -(y / (1 + np.exp(y * score))) * x


                self.w = self.w - eta*gradient
                self.w = (1-eta * self.lambda_value) * self.w


pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    Log_reg()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))
Log_reg_time = t1-t0

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.83 sec.
Accuracy: 0.8082.


## Bonus Task 1:
### a) Making it faster with BLAS (or, making it BLASing fast!)

In [4]:
from scipy.linalg import blas
class BLASLinearClassifier(LinearClassifier):
    def decision_function(self, X: np.array):
        return blas.ddot(X, self.w)

class BLASSVM(LinearClassifier):
    def __init__(self, lambda_value=0.001, n_iter=20) -> None:
        self.n_iter = n_iter
        self.lambda_value = lambda_value

    def fit(self, X, Y):
        self.find_classes(Y)

        Ye = self.encode_outputs(Y)
    
        if not isinstance(X, np.ndarray):
            X = X.toarray()
        
        self.w = np.zeros(X.shape[1])
        t = 0

        #Pegasos
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                t = t + 1

                eta = 1/(self.lambda_value * t)
                score = blas.ddot(self.w, x) 

                if (y * score) < 1:
                    # self.w = self.w * (1-eta * self.lambda_value) + ((eta*y) * x)
                    blas.dscal((1-eta * self.lambda_value), self.w)
                    blas.daxpy(((eta*y) * x), self.w, a=1)

                else:
                    blas.dscal((1-eta * self.lambda_value), self.w)

pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    BLASSVM()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print(f'Training time: {round(t1-t0, 3)} sec, without BLAS it took {round(SVM_time, 3)}')

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.395 sec, without BLAS it took 1.469
Accuracy: 0.8238.


### b) There is Nothing to see here. Making it sparse

In [5]:
def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)

class SparseSVM(LinearClassifier):
    def __init__(self, lambda_value=0.001, n_iter=20) -> None:
        self.n_iter = n_iter
        self.lambda_value = lambda_value

    def fit(self, X, Y):
        self.find_classes(Y)

        Ye = self.encode_outputs(Y)
        
        self.w = np.zeros(X.shape[1])
        t = 0

        #Pegasos
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                # self.w <class 'numpy.ndarray'> /Dense matrix
                t = t + 1

                eta = 1/(self.lambda_value * t)
                score = sparse_dense_dot(x, self.w)

                if (y * score) < 1:
                    #self.w = (1-eta * self.lambda_value) * self.w + (eta*y) * x
                    add_sparse_to_dense(eta*y*x, self.w, factor=(1-eta * self.lambda_value))
                else:
                    self.w = (1-eta * self.lambda_value) * self.w    #This part cannot be sparse-dense functioned with what we have. Would need a... sparse_dense_scale?


pipeline = make_pipeline(
    TfidfVectorizer(),
    #SelectKBest(k=1000),
    Normalizer(),

    SVM()  
)
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
SVM_time = t1-t0

pipeline = make_pipeline(
    TfidfVectorizer(),
    #SelectKBest(k=1000),
    Normalizer(),

    SparseSVM()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
SparseSVM_time = t1-t0
print(f'Training time: {round(SparseSVM_time, 3)} sec, without sparse matricies it took {round(SVM_time, 3)}')

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


Epoch 0
Score: -1.20
Epoch 5
Score: -1.05
Epoch 10
Score: -1.02
Epoch 15
Score: -1.01
Training time: 8.606 sec, without sparse matricies it took 38.424
Accuracy: 0.8288.


### c) "Scaling up the speed of our operations was a real 'byte' in the pants, but it was worth it for the 'mega' improvements."

In [6]:
class SparseSVM(LinearClassifier):
    def __init__(self, lambda_value=0.001, n_iter=20) -> None:
        self.n_iter = n_iter
        self.lambda_value = lambda_value

    def fit(self, X, Y):
        self.find_classes(Y)

        Ye = self.encode_outputs(Y)
        
        self.w = np.zeros(X.shape[1])
        t = 0
        a = 1

        #Pegasos
        for i in range(self.n_iter):
            for x_i, y_i in zip(X, Ye):
                t = t + 1

                eta = 1/(self.lambda_value * t)
                score = a * sparse_dense_dot(x_i, self.w)

                if (y_i * score) < 1:
                    #self.w = (1-eta * self.lambda_value) * self.w + (eta*y) * x
                    add_sparse_to_dense(((eta* y_i)/a) * x_i, self.w, factor=(1-eta * self.lambda_value))
                else:
                    a = (1-eta*self.lambda_value)*a
        self.w = self.w * a 

pipeline = make_pipeline(
    TfidfVectorizer(),
    #SelectKBest(k=1000),
    Normalizer(),

    SparseSVM()  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print(f'Training time: {round(t1-t0, 3)} sec, without changed scaling it took {round(SparseSVM_time, 3)}')

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 7.717 sec, without changed scaling it took 8.606
Accuracy: 0.8288.
